In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

# Make sure you have mounted your Google Drive before running this cell.
# from google.colab import drive
# drive.mount('/content/drive')

# Replace the file paths with the actual paths to your files in Google Drive
try:
    df1= pd.read_csv('/content/drive/My Drive/project/POS_CASH_balance.csv')
    df2= pd.read_csv('/content/drive/My Drive/project/application_test.csv')
    df3= pd.read_csv('/content/drive/My Drive/project/application_train.csv')
    df4= pd.read_csv('/content/drive/My Drive/project/previous_application.csv')
    df5= pd.read_csv('/content/drive/My Drive/project/bureau.csv')
    df6= pd.read_csv('/content/drive/My Drive/project/bureau_balance.csv')
    df7= pd.read_csv('/content/drive/My Drive/project/credit_card_balance.csv')
    df8= pd.read_csv('/content/drive/My Drive/project/installments_payments.csv')
    print("All files read successfully!")
except FileNotFoundError as e:
    print(f"Error: {e}. Please check the file paths and ensure the files exist in your Google Drive.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


#JUST CHECKING THE REPORT FOR THE MAIN COLUMN WITH A LOT OF SETS
profile = ProfileReport(df1, title="Pandas Profiling Report")
profile.to_file(output_file = 'output1.html')


sample = [df1, df2, df3, df4, df5, df6, df7, df8]
names =['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8']

for name,df in zip(names,sample):
   corr = df.corrwith(df3["TARGET"], numeric_only=True)
   print(f"Correlation matrix for {name}:")
   print(corr)


sample = [df1, df2, df3, df4, df5, df6, df7, df8]
names = ['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8']

for name, df in zip(names, sample):
    for column in df.columns:
        null_count = df[column].isnull().sum()
        percentage = (null_count / df.shape[0]) * 100  # Percentage of nulls in this column
        print(f"Percentage of null values in {name}, column '{column}': {percentage:.2f}%")


sample = [df1, df2, df3, df4, df5, df6, df7, df8]
names =['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8']
for name,df in zip(names,sample):
 print(f"\nCount of each data type in {name} f1:")
 display(df.dtypes.value_counts())


# ALL work first on bureau_application
bureau_balance_aggregate = df6.groupby("SK_ID_BUREAU").agg({ "MONTHS_BALANCE": ["min", "max", "mean"] })
bureau_balance_aggregate.columns = ["BB_" + "_".join(col).upper() for col in bureau_balance_aggregate.columns]
bureau_balance_aggregate.reset_index(inplace=True)
df5 = df5.merge(bureau_balance_aggregate, on="SK_ID_BUREAU", how="left")

#ALL CHANGES NOW TO BE DONE IN BUREAU CSV
df5_agg = df5.drop(columns=['SK_ID_CURR', 'SK_ID_BUREAU'])

# Select only numeric columns
numeric_cols_df5 = df5_agg.select_dtypes(include='number')


agg_dict_df5 = {}
for col in numeric_cols_df5.columns:
    agg_dict_df5[col] = ['mean', 'max', 'min']

# Aggregate df5
bureau_aggregate_numeric = df5.groupby("SK_ID_CURR").agg(agg_dict_df5)

# Flatten the multi-level column names for numeric aggregation
bureau_aggregate_numeric.columns = ["_".join(col).strip() for col in bureau_aggregate_numeric.columns.values]

# Reset index
bureau_aggregate_numeric.reset_index(inplace=True)


non_numeric_cols_df5 = df5_agg.select_dtypes(exclude='number')

# Define the aggregation dictionary for non-numeric columns using mode
agg_dict_non_numeric_df5 = {}
for col in non_numeric_cols_df5.columns:
    agg_dict_non_numeric_df5[col] = lambda x: x.mode()[0] if not x.mode().empty else None

# Aggregate non-numeric columns by SK_ID_CURR
bureau_aggregate_non_numeric = df5.groupby("SK_ID_CURR").agg(agg_dict_non_numeric_df5)

# Reset index to make SK_ID_CURR a column
bureau_aggregate_non_numeric.reset_index(inplace=True)

# Merge
bureau_aggregate_all = pd.merge(bureau_aggregate_numeric, bureau_aggregate_non_numeric, on='SK_ID_CURR', how='left')


#ALL ON CREDIT CARD BALANCE NOW
for column in df7.columns:
  null_count = df7[column].isnull().sum()
  percentage = (null_count/df7.shape[0])*100
  print(f"null percentage of {column} is equal to {percentage:.2f}%")


credit_install_agg = df7.groupby("SK_ID_CURR").agg({
    "MONTHS_BALANCE": ["min", "max", "mean"],
    "AMT_BALANCE": ["sum", "mean", "max"],
    "AMT_CREDIT_LIMIT_ACTUAL": ["mean", "max"],
    "SK_DPD": ["max", "mean"],
    "CNT_DRAWINGS_ATM_CURRENT": "sum",
    "CNT_DRAWINGS_CURRENT": "sum",
    "CNT_DRAWINGS_OTHER_CURRENT": "sum",
    "CNT_DRAWINGS_POS_CURRENT": "sum",
    "AMT_DRAWINGS_ATM_CURRENT": "sum",
    "AMT_DRAWINGS_CURRENT": "sum",
    "AMT_DRAWINGS_OTHER_CURRENT": "sum",
    "AMT_DRAWINGS_POS_CURRENT": "sum"
})

credit_install_agg['SUM_ALL_CNT_DRAWINGS'] = credit_install_agg[[('CNT_DRAWINGS_ATM_CURRENT', 'sum'),
                                                   ('CNT_DRAWINGS_CURRENT', 'sum'),
                                                   ('CNT_DRAWINGS_OTHER_CURRENT', 'sum'),
                                                   ('CNT_DRAWINGS_POS_CURRENT', 'sum')]].sum(axis=1)

credit_install_agg['SUM_ALL_AMT_DRAWINGS'] = credit_install_agg[[('AMT_DRAWINGS_ATM_CURRENT', 'sum'),
                                                   ('AMT_DRAWINGS_CURRENT', 'sum'),
                                                   ('AMT_DRAWINGS_OTHER_CURRENT', 'sum'),
                                                   ('AMT_DRAWINGS_POS_CURRENT', 'sum')]].sum(axis=1)

install_agg = df8.groupby("SK_ID_CURR").agg({
    "NUM_INSTALMENT_VERSION": ["nunique"],
    "AMT_INSTALMENT": ["sum", "mean", "max"],
    "AMT_PAYMENT": ["sum", "mean", "min"],
    "DAYS_ENTRY_PAYMENT": ["min", "max", "mean"]})

POS_agg = df1.groupby("SK_ID_CURR").agg({'MONTHS_BALANCE': ['max', 'mean'],
        'SK_DPD': ['max', 'mean', 'sum'],
        'SK_DPD_DEF': ['max', 'mean', 'sum'],
        'CNT_INSTALMENT_FUTURE': ['mean', 'sum'],
        'CNT_INSTALMENT': ['max'],
        'NAME_CONTRACT_STATUS': lambda x: x.mode()[0] if not x.mode().empty else None})

prev_application_agg = df4_copy.groupby("SK_ID_CURR").agg({


        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'DAYS_DECISION': ['min', 'max', 'mean'],   })

df_train = df3.copy()
df_test = df2.copy()
df_train = df_train.merge(bureau_aggregate_all, on="SK_ID_CURR", how="left")
df_test  = df_test.merge(bureau_aggregate_all, on="SK_ID_CURR", how="left")

credit_install_agg.columns = ["_".join(col).strip() for col in credit_install_agg.columns.values]

df_train = df_train.merge(credit_install_agg, on="SK_ID_CURR", how="left")
df_test  = df_test.merge(credit_install_agg, on="SK_ID_CURR", how="left")


install_agg.columns = ["_".join(col).strip() for col in install_agg.columns.values]

df_train = df_train.merge(install_agg, on="SK_ID_CURR", how="left")
df_test  = df_test.merge(install_agg, on="SK_ID_CURR", how="left")


POS_agg.columns = ["_".join(col).strip() for col in POS_agg.columns.values]

df_train = df_train.merge(POS_agg, on="SK_ID_CURR", how="left")
df_test  = df_test.merge(POS_agg, on="SK_ID_CURR", how="left")


prev_application_agg.columns = ["_".join(col).strip() for col in prev_application_agg.columns.values]

df_train = df_train.merge(prev_application_agg, on="SK_ID_CURR", how="left")
df_test  = df_test.merge(prev_application_agg, on="SK_ID_CURR", how="left")

# FROM HERE ON FORGET ABT THE TEST THING FOR A WHILE I AM JUST TESTING THE DATA HERE, SO DO NOT FORGET
 #  TO MAKE THE SAME CHANGES IN TRAIN LATER FROM THE CODE USED HERE ON NOW
 # DONT FORGET
 # DONT FORGET
 # DONT FORGET THIS IS THE LINE
 # CHECK IF YOU HAVE ENCODED THE ORIGINAL TEST AND TRAIN


X = df_train.drop(columns=["TARGET"])
y = df_train["TARGET"]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y )


label_encoders = {}
for col in X_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    # Apply the same encoder
    X_valid[col] = le.transform(X_valid[col].astype(str))
    label_encoders[col] = le


#GRID SEARCH RIGHT NOW
param_grid = {
    "learning_rate": [ 0.03, 0.05],
    "num_leaves": [32, 64],
    "feature_fraction": [0.8],
    "subsample": [0.8],
    "random_state": [42,62],
    "reg_alpha": [1.0, 0.4],  # L1 regularization
    "reg_lambda": [1.0, 0.4], # L2 regularization
}


lgbm = lgb.LGBMClassifier(objective='binary', metric='auc')


grid = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=1,
    n_jobs=-1  )


grid.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid.best_params_)
print("Best ROC AUC score (CV):", grid.best_score_)

# Evaluate on validation set (optional)
y_pred = grid.best_estimator_.predict_proba(X_valid)[:, 1]
roc_auc = roc_auc_score(y_valid, y_pred)
print("Validation ROC AUC:", roc_auc)


# INITIAL MODEL THIS IS
params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 32,
    "feature_fraction": 0.8,
    "subsample": 0.8,
    "random_state": 42,
    "lambda_l1": 1.0,  # L1 regularization
    "lambda_l2": 1.0   # L2 regularization
}

# Prepare LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

# Train the model with early stopping THIS REMEMBER WILL BE HELPFUL TOOK TIME
model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)]
)

# Predict probabilities for the validation set
y_pred = model.predict(X_valid)

# Calculate ROC AUC
roc_auc = roc_auc_score(y_valid, y_pred)
print("Validation ROC AUC:", roc_auc)


#CHECKING THE IMPORTANCE OF EACH

importances = model.feature_importance()
feature_names = X_train.columns


importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})


importance_df = importance_df.sort_values(by='importance', ascending=False)

print(importance_df)


importance_df['percentage'] = importance_df['importance'] / importance_df['importance'].sum() * 100
importance_df.head()


from os import PRIO_PGRP
print(importance_df['importance'].sum())
filt = importance_df['percentage']<2
print(importance_df.loc[filt].count())


#THIS MEANS ALL COLUMNS ARE MORE OR LESS IMPORTANT AS ALL COLUMNS HAVE IMPORTANCE LESS THAN 6 PERCENT
#HERE ON DOING IT FOR THE TEST COLUMN
label_encoders = {}
for col in df_test.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_test[col] = le.fit_transform(df_test[col].astype(str))
    label_encoders[col] = le  # Optionally save the encoder for inverse_transform later


label_encoders = {}
for col in df_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col].astype(str))
    label_encoders[col] = le



Xtrainfin = df_train.drop(columns=['TARGET'])
Ytrainfin = df_train['TARGET']

import lightgbm as lgb


model2 = lgb.LGBMClassifier(
    objective="binary",
    metric="auc",
    learning_rate=0.05,
    num_leaves=32,
    feature_fraction=0.8,
    subsample=0.8,
    random_state=42,
    reg_alpha=1.0,   # L1 regularization
    reg_lambda=1.0   # L2 regularization
)

# Fit the model to your training data
model2.fit(Xtrainfin, Ytrainfin)


y_test_pred = model2.predict_proba(df_test)[:, 1]
output = pd.DataFrame({'SK_ID_CURR': df_test['SK_ID_CURR'], 'TARGET': y_test_pred})
output.to_csv('submission.csv', index=False)